#### □ 사전준비 □

1) Google 검색출력화면의 이해필요

2) 검색결과창에서 F12로 구성내역 확인 → 각 요소의 attribute 명칭, class 명칭 파악

※ 주의 - 과거이력을 볼 때, 구글 맘대로 요소 attribute 내역이나 class 명칭을 바꾸거나 파서가 바뀔 수도 있는 것 같음 (지꺼니까 당연한가;;)

#### ■ 개괄설명 ■

1) '검색어(Keywords)' 및 '조회페이지 수' 임의입력

2) selenium 라이브러리 內 webdriver 활용한 브라우저(크롬) 제어

3) page_source확보후 Beautifulsoup로 파싱(parcing)

4) 사전 살펴본  attribute 명칭, class 명칭으로 원하는 요소 데이터를 찾음

5) 보고싶은 자료 형태로 가공

#### ■ 코딩내역 ■ (잡설포함)

In [ ]:
# 라이브러리 설치 명령어
! pip install requests
! pip install selenium
! pip install bs4
! pip install openpyxl
! pip install pandas
! pip install numpy
! pip install re

In [12]:
""" 관련 라이브러리 목록 """
#-*- coding:utf-8 -*-  # 한글 안깨질려고 utf-8 인코딩
# Crawling 관련 라이브러리
import urllib.request
from urllib import parse
from bs4 import BeautifulSoup
from selenium import webdriver

# 데이터 핸들링 관련 라이브러리
import time
import pandas as pd
import numpy as np
import re

""" 검색어(≒키워드) 관리 """
keywords = ['페코린느'] # , '고구마' , '호박']

""" 조회 페이지 조정 """
number_of_pages = 2

""" 결과삽입용 데이터 프레임 미리 만들어 놓음(여기에 keywords에 따른 결과를 하나하나 넣을 것임) """
result= pd.DataFrame([])

""" 크롤링 """
# 크롤링은 kewords 리스트에 있는 단어들을 하나하나 수행하는게 필요. 
# kewords 리스트에 '페코린느' 하나만 있으면 한 번 수행하겠지만? 여러개면 kewords 리스트 안에껄 전부완료 할 때까지 반복작업해야함
# 또한 kewords 별로, 구글 검색결과 '몇' 페이지까지 조회하는지 설정두 해야함. 그러니까 단어 하나하나, 설정한 조회페이지수 마지막까지 들여다 보게해야함
# 그래서 반복작업이 필요하니, for 문으로 loop를 돌리는 거고,
# 단어수 * 조회페이지수 두 가지 기준으로 반복되니, for문을 두개 겹쳐 쓴 것

for keyword in keywords: # keywords 리스트 안에서, keyword 하나 선택
    for number in range(0, number_of_pages): # number_of_pages 결정하면 페이지 수대로 검색
        
        # keyword 문자 그대로 url에 인풋
        url = 'https://www.google.com/search?q={}&start={}'.format(keyword , int(number*10)) # 예 : keywords = '프리코네'

        # keyword 문자를 url 인코딩해서 인풋 
        # 혹시나 나중에라도,사이트+검색어 부분에서 검색어세 인코딩된 내역을 요구할까봐 남겨둠
        # keyword_encoded = parse.quote(keyword)
        # url = 'https://www.google.com/search?q={}&start={}'.format(keyword , int(number*10)) # 예 : keywords = '%ED%94%84%EB%A6%AC%EC%BD%94%EB%84%A4'

        ######################################################################################
        ######################################################################################
        """ 주의 : 하기 Chorm() 괄호에는 반드시 자신 컴퓨터의 chrome 깔린 위치를 입력해줘야함 """
        # webdriver 란 selenium 클래스한테, Chrome 사용지시
        driver = webdriver.Chrome('C:/Users/amore/Downloads/chromedriver_win32/chromedriver')  # 반드시 자신 컴퓨터의 크롬 깔린 폴더경로 입력
        ######################################################################################
        ######################################################################################

        # 크롬먹은 webdriver(이하 driver)에게 url 꽂아주면서 켜라고 지시 (사이트+검색어 내역을 꽂음)
        driver.get(url)

        # driver 한테서 page_source를 뱉어내게하고, 
        # BeatifulSoup 보고 html 코드니까 해석하라고 시킴
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 구글 검색페이지 2020-03-31 기준,
        # 사이트 이름의 경우, attribute 이름 h3 로
        # 광고일 경우 class 가 sA5rQ , 일반페이지일경우 class 가 LC20lb DKV0Md
        # 사이트 주소의 경우, arrtibute 이름 div로
        # 광고일 경우 class 가 ad_cclk , 일반페이지일경우 class 가 r
        # 이거 추후에 바뀔 수도 있으므로 주의
        total_names = soup.find_all('h3', {"class": ["sA5rQ", "LC20lb DKV0Md"]})
        total_links = soup.find_all('div', {"class": ["ad_cclk", "r"]})

        # 음..이하는 그냥.. 노가다 작업.
        # 구글이 코딩해놓은 사이트 규칙에 맞게 필요한 정보를 추출함
        # 추출하는건 re 라이브러리 findall 함수를 썼음

        # 광고페이지냐 일반페이지냐 솔팅
        total_names_list = [''.join(re.findall(r'(<h3 class="LC20lb DKV0Md">.*|<h3 class="sA5rQ">.*)', item )) for item in str(total_names)[1:-1].split('</h3>, ') ]
        Separator = [item.replace('<h3 class="LC20lb DKV0Md">', '일반 ') for item in total_names_list]
        Separator = [item.replace('<h3 class="sA5rQ">', '광고 ') for item in Separator]
        Separator = [ x for x in Separator if ("일반 " in x) | ("광고 " in x)]
        Separator= [item[:2] for item in Separator]

        # 사이트 이름 추출
        total_names_list = [item.replace('<h3 class="LC20lb DKV0Md">', '') for item in total_names_list]
        total_names_list = [item.replace('<h3 class="sA5rQ">', '') for item in total_names_list]
        total_names_list = [item.replace('\u200e', '') for item in total_names_list]
        total_names_list = [item.replace('</h3>', '') for item in total_names_list]

        # 사이트 링크 추출
        # 흠.. 광고의 경우, 사이트를 두개를 가지고 있어서 ( 하나는 data-preconnect-url , 하나는 href) 혹시 몰라냄겨둠
        # total_links_list = [''.join(re.findall(r'(data-preconnect-urls=.*"|href=.*)', item )) for item in str(total_links).replace('https','http').split(' ')]
        total_links_list = [''.join(re.findall(r'(href=.*)', item )) for item in str(total_links).replace('https','http').split(' ')]
        total_links_list = [ x for x in total_links_list if "webcache" not in x ]
        # total_links_list = [ x for x in total_links_list if 'href="/aclk?sa' not in x ]  # 얘는 그냥 기준으로 못쓰겠다.. 
        total_links_list = [ x for x in total_links_list if 'amp;adurl="' not in x ]
        total_links_list = [ x for x in total_links_list if 'href="/search?q=related' not in x ]
        total_links_list = list(filter(lambda a: (a != '')&(a != 'href="#"'), total_links_list))

        # href= 구분자로 물고 온지라, 앞에서 떼어냄, 앞뒤의 "" 도 삭제
        total_links_list = [ item[6:-1] for item in total_links_list ]

        # 임시표 temp 작성
        temp = pd.DataFrame([])
        temp['keywords'] = '' # Separator 나 사이트이름이나, 주소들은 모두 여러개인데, keywords는 하나니까 자리만 뚫어놓고
        temp['구분'] = Separator
        temp['노출순위\n(광고포함)'] = ''
        temp['노출순위\n(광고제외)'] = ''
        temp['표시페이지'] = number+1
        temp['사이트명'] = total_names_list
        temp['사이트주소'] = total_links_list
        temp['노출도평가'] = ''
        temp['keywords'] = keyword # 자리가 다 들어차면 keywords 이름 던져줌

        # 미리 만들어논 result 에 temp 삽입
        result = pd.concat([result, temp] , axis=0)

        # temp 토사구팽
        del temp
        
        # 크롤링 다했으면 프-바
        driver.close()

    """ 순위부여 """
    result['노출순위\n(광고포함)'] = list(range(1 , result.shape[0]+1))
    result['노출순위\n(광고제외)'] = np.where( result['구분'] == '일반' , 1 , 0 )
    result['노출순위\n(광고제외)'] =result['노출순위\n(광고제외)'].cumsum()
    result['노출순위\n(광고제외)'] = np.where( result['구분'] == '일반' ,result['노출순위\n(광고제외)'] , '' )
    
    """ 노출도평가 """
    # 1 페이지 3위이내 -> 검색어 친 후, 유저 가시권해당 -> 최상
    # 1 페이지 하위 3등 -> 검색어 친 후, 굳이 스크롤 내려야 보임 -> 중
    # 1 페이지 중간층 -> 검색어 친 후, 스윽 하면 보임 -> 상
    # 1 페이지 아님 -> 검색어 친 후, 굳이 수고를 들여 클릭해야함, 
    #                  즉 클릭하는 사용자는'뭔가'를 찾는 니즈를 이미 가지고 있음. 2페이지 노출된다고 중? 이건 아닌거같음 -> 하
    result['노출순위\n(광고포함)'] = result['노출순위\n(광고포함)'].astype(int)
    result['노출도평가'] = '하'
    result['노출도평가'] = np.where( result['노출순위\n(광고포함)'] <= 3, '최상', 
                           np.where( (result['노출순위\n(광고포함)'] >= result[result['표시페이지'] == 1].shape[0]-2) & (result['표시페이지']==1), '중',
                           np.where(  result['표시페이지'] != 1 , '하' , '상')))
    
""" 파일저장 """
""" 아래 to_excel() 요기에 엑셀이 저장될 위치를 넣어줘야함  """
result.to_excel('C:/Users/amore/Desktop/test.xlsx' , # ◀ ★(중요)★ excel 파일 저장할 위치 설정
                header = True, index = False,  # 헤더는 ok , 인덱스는 노노해
                startrow = 1, startcol = 1, # 1행 1열부터 데이터 푸셈
                freeze_panes = (2, 0))  # 2행부터 행고정 거셈